# import 문

In [1]:
import torch
import glob
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from tqdm import tqdm
import numpy as np
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
import time
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
from torch_poly_lr_decay import PolynomialLRDecay
import random


torch.set_num_threads(1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data 불러오기

In [3]:
labels_df = pd.read_csv('data/dirty_mnist_2nd_answer.csv')[:]
imgs_dir = np.array(sorted(glob.glob('data/dirty_mnist_2nd/*')))[:]
labels = np.array(labels_df.values[:,1:])

In [4]:
test_imgs_dir = np.array(sorted(glob.glob('data/test_dirty_mnist_2nd/*')))

In [4]:
imgs=[]

In [ ]:
for path in tqdm(imgs_dir[:]):
    img=cv2.imread(path, cv2.IMREAD_COLOR)
    imgs.append(img)

 39%|███▉      | 19626/50000 [00:14<00:22, 1344.93it/s]

In [ ]:
imgs=np.array(imgs)

# 데이터 클래스 정의

In [5]:
class MnistDataset_v1(Dataset):
    def __init__(self, imgs_dir=None, labels=None, transform=None, train=True):
        self.imgs_dir = imgs_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # 1개 샘플 get
        img = cv2.imread(self.imgs_dir[idx], cv2.IMREAD_COLOR)
        img = self.transform(img)
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
        pass
    


# 메모리에서 load
class MnistDataset_v2(Dataset):
    def __init__(self, imgs=None, labels=None, transform=None, train=True):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        self.train=train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # 1개 샘플 get1
        img = self.imgs[idx]
        img = self.transform(img)
        
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img

## 시드 고정

In [6]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  

# 모델 정의

In [7]:
class EfficientNet_MultiLabel(nn.Module):
    def __init__(self, in_channels):
        super(EfficientNet_MultiLabel, self).__init__()
        self.network = EfficientNet.from_pretrained('efficientnet-b7', in_channels=in_channels)
        self.output_layer = nn.Linear(1000, 26)

    def forward(self, x):
        x = F.relu(self.network(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# K-Fold

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in kf.split(imgs):
    folds.append((train_idx, valid_idx))

# 모델 훈련

In [11]:
seed_everything(42)

# 5개의 fold 모두 실행하려면 for문을 5번 돌리면 됩니다.
for fold in range(5):
    model = EfficientNet_MultiLabel(in_channels=3).to(device)
#     model = nn.DataParallel(model)
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]



    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor()
        ])
    valid_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
        ])


    epochs=50
    batch_size=40         # 자신의 VRAM에 맞게 조절해야 OOM을 피할 수 있습니다.
    
    
    
    # Data Loader
    train_dataset = MnistDataset_v2(imgs = imgs[train_idx], labels=labels[train_idx], transform=train_transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = MnistDataset_v2(imgs = imgs[valid_idx], labels = labels[valid_idx], transform=valid_transform)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)       
    
    
    # optimizer
    # polynomial optimizer를 사용합니다.
    # 
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
    decay_steps = (len(train_dataset)//batch_size)*epochs
    scheduler_poly_lr_decay = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)

    criterion = torch.nn.BCELoss()
    
    
    epoch_accuracy = []
    valid_accuracy = []
    valid_losses=[]
    valid_best_accuracy=0
    for epoch in range(epochs):
        model.train()
        batch_accuracy_list = []
        batch_loss_list = []
        start=time.time()
        for n, (X, y) in enumerate((train_loader)):
            X = torch.tensor(X, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            y_hat = model(X)
            
            
            optimizer.zero_grad()
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            scheduler_poly_lr_decay.step()

            
            y_hat  = y_hat.cpu().detach().numpy()
            y_hat = y_hat>0.5
            y = y.cpu().detach().numpy()

            batch_accuracy = (y_hat == y).mean()
            batch_accuracy_list.append(batch_accuracy)
            batch_loss_list.append(loss.item())

        model.eval()
        valid_batch_accuracy=[]
        valid_batch_loss = []
        with torch.no_grad():
            for n_valid, (X_valid, y_valid) in enumerate((valid_loader)):
                X_valid = torch.tensor(X_valid, device=device)#, dtype=torch.float32)
                y_valid = torch.tensor(y_valid, device=device, dtype=torch.float32)
                y_valid_hat = model(X_valid)
                
                valid_loss = criterion(y_valid_hat, y_valid).item()
                
                y_valid_hat = y_valid_hat.cpu().detach().numpy()>0.5
                
                
                valid_batch_loss.append(valid_loss)
                valid_batch_accuracy.append((y_valid_hat == y_valid.cpu().detach().numpy()).mean())
                
            valid_losses.append(np.mean(valid_batch_loss))
            valid_accuracy.append(np.mean(valid_batch_accuracy))
            
        if np.mean(valid_batch_accuracy)>valid_best_accuracy:
            torch.save(model.state_dict(), 'model/EfficientNetB7-fold{}.pt'.format(fold))
            valid_best_accuracy = np.mean(valid_batch_accuracy)
        print('fold : {}\tepoch : {:02d}\ttrain_accuracy / loss : {:.5f} / {:.5f}\tvalid_accuracy / loss : {:.5f} / {:.5f}\ttime : {:.0f}'.format(fold+1, epoch+1,
                                                                                                                                              np.mean(batch_accuracy_list),
                                                                                                                                              np.mean(batch_loss_list),
                                                                                                                                              np.mean(valid_batch_accuracy), 
                                                                                                                                              np.mean(valid_batch_loss),
                                                                                                                                              time.time()-start))

Loaded pretrained weights for efficientnet-b7


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(Tru

fold : 1	epoch : 01	train_accuracy / loss : 0.60568 / 0.65019	valid_accuracy / loss : 0.64088 / 0.62710	time : 939
fold : 1	epoch : 02	train_accuracy / loss : 0.69713 / 0.56847	valid_accuracy / loss : 0.72633 / 0.54007	time : 938
fold : 1	epoch : 03	train_accuracy / loss : 0.76686 / 0.48520	valid_accuracy / loss : 0.78390 / 0.46098	time : 938
fold : 1	epoch : 04	train_accuracy / loss : 0.81220 / 0.41767	valid_accuracy / loss : 0.80231 / 0.43665	time : 938
fold : 1	epoch : 05	train_accuracy / loss : 0.84272 / 0.36542	valid_accuracy / loss : 0.82554 / 0.39381	time : 938
fold : 1	epoch : 06	train_accuracy / loss : 0.86460 / 0.32408	valid_accuracy / loss : 0.85897 / 0.33466	time : 937
fold : 1	epoch : 07	train_accuracy / loss : 0.88159 / 0.28980	valid_accuracy / loss : 0.86797 / 0.31801	time : 936
fold : 1	epoch : 08	train_accuracy / loss : 0.89496 / 0.26172	valid_accuracy / loss : 0.87750 / 0.30054	time : 936
fold : 1	epoch : 09	train_accuracy / loss : 0.90605 / 0.23672	valid_accuracy / l

fold : 2	epoch : 22	train_accuracy / loss : 0.97068 / 0.08094	valid_accuracy / loss : 0.92283 / 0.23998	time : 937
fold : 2	epoch : 23	train_accuracy / loss : 0.97371 / 0.07318	valid_accuracy / loss : 0.92200 / 0.25055	time : 936
fold : 2	epoch : 24	train_accuracy / loss : 0.97630 / 0.06635	valid_accuracy / loss : 0.89860 / 0.32059	time : 936
fold : 2	epoch : 25	train_accuracy / loss : 0.97884 / 0.05940	valid_accuracy / loss : 0.91573 / 0.29738	time : 937
fold : 2	epoch : 26	train_accuracy / loss : 0.98078 / 0.05425	valid_accuracy / loss : 0.92437 / 0.27425	time : 937
fold : 2	epoch : 27	train_accuracy / loss : 0.98293 / 0.04846	valid_accuracy / loss : 0.92347 / 0.27465	time : 937
fold : 2	epoch : 28	train_accuracy / loss : 0.98436 / 0.04431	valid_accuracy / loss : 0.92615 / 0.27052	time : 937
fold : 2	epoch : 29	train_accuracy / loss : 0.98622 / 0.03957	valid_accuracy / loss : 0.92683 / 0.28637	time : 937
fold : 2	epoch : 30	train_accuracy / loss : 0.98756 / 0.03578	valid_accuracy / l

fold : 3	epoch : 43	train_accuracy / loss : 0.99734 / 0.00794	valid_accuracy / loss : 0.93189 / 0.39862	time : 938
fold : 3	epoch : 44	train_accuracy / loss : 0.99776 / 0.00667	valid_accuracy / loss : 0.93351 / 0.38942	time : 938
fold : 3	epoch : 45	train_accuracy / loss : 0.99809 / 0.00580	valid_accuracy / loss : 0.93421 / 0.39781	time : 938
fold : 3	epoch : 46	train_accuracy / loss : 0.99846 / 0.00480	valid_accuracy / loss : 0.93460 / 0.42335	time : 938
fold : 3	epoch : 47	train_accuracy / loss : 0.99864 / 0.00426	valid_accuracy / loss : 0.93596 / 0.41536	time : 938
fold : 3	epoch : 48	train_accuracy / loss : 0.99887 / 0.00345	valid_accuracy / loss : 0.93660 / 0.43268	time : 938
fold : 3	epoch : 49	train_accuracy / loss : 0.99915 / 0.00273	valid_accuracy / loss : 0.93680 / 0.43249	time : 938
fold : 3	epoch : 50	train_accuracy / loss : 0.99921 / 0.00240	valid_accuracy / loss : 0.93689 / 0.44734	time : 938
Loaded pretrained weights for efficientnet-b7
fold : 4	epoch : 01	train_accuracy

fold : 5	epoch : 14	train_accuracy / loss : 0.94118 / 0.15607	valid_accuracy / loss : 0.91219 / 0.23693	time : 937
fold : 5	epoch : 15	train_accuracy / loss : 0.94613 / 0.14380	valid_accuracy / loss : 0.91587 / 0.22908	time : 937
fold : 5	epoch : 16	train_accuracy / loss : 0.95035 / 0.13281	valid_accuracy / loss : 0.85659 / 0.39250	time : 937
fold : 5	epoch : 17	train_accuracy / loss : 0.95468 / 0.12221	valid_accuracy / loss : 0.90452 / 0.26883	time : 937
fold : 5	epoch : 18	train_accuracy / loss : 0.95912 / 0.11149	valid_accuracy / loss : 0.76577 / 0.61140	time : 937
fold : 5	epoch : 19	train_accuracy / loss : 0.96274 / 0.10208	valid_accuracy / loss : 0.85508 / 0.39954	time : 937
fold : 5	epoch : 20	train_accuracy / loss : 0.96622 / 0.09317	valid_accuracy / loss : 0.91998 / 0.24458	time : 937
fold : 5	epoch : 21	train_accuracy / loss : 0.96938 / 0.08449	valid_accuracy / loss : 0.91288 / 0.27127	time : 937
fold : 5	epoch : 22	train_accuracy / loss : 0.97235 / 0.07680	valid_accuracy / l

# 테스트 데이터 input type으로

In [8]:
test_imgs=[]
for path in tqdm(test_imgs_dir):
    test_img=cv2.imread(path, cv2.IMREAD_COLOR)
    test_imgs.append(test_img)
test_imgs=np.array(test_imgs)
seed_everything(42)

100%|██████████| 5000/5000 [00:03<00:00, 1318.62it/s]


In [9]:
import ttach as tta
test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        ])

tta_transforms = tta.Compose(
    [
        tta.VerticalFlip(),
        tta.HorizontalFlip(),
        tta.Rotate90(angles=[0, 90, 180, 270]),        
    ]
)

# 제출 파일 생성

In [10]:
submission = pd.read_csv('data/sample_submission.csv')

with torch.no_grad():
    for fold in range(5):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('model/EfficientNetB7-fold{}.pt'.format(fold)))
        tta_model = tta.ClassificationTTAWrapper(model, tta_transforms)
        tta_model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            for n_t, tta_transforms_0 in enumerate(tta_transforms):
                tta_img = tta_transforms_0.augment_image(X_test)
                tta_model.eval()
                model_output = tta_model(tta_img)
                if (n_t == 0):
                    model_outputs = (model_output.to(torch.device("cpu"))).numpy()
                else:
                    model_outputs = model_outputs + (model_output.to(torch.device("cpu"))).numpy()
                
            pred_test = model_outputs / n_t
            submission.iloc[n*32:(n+1)*32,1:] += pred_test

Loaded pretrained weights for efficientnet-b7


  0%|          | 0/157 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 157/157 [1:54:35<00:00, 35.06s/it]


Loaded pretrained weights for efficientnet-b7


100%|██████████| 157/157 [1:54:35<00:00, 34.98s/it]


Loaded pretrained weights for efficientnet-b7


100%|██████████| 157/157 [1:54:35<00:00, 34.98s/it]


Loaded pretrained weights for efficientnet-b7


100%|██████████| 157/157 [1:54:33<00:00, 34.97s/it]


Loaded pretrained weights for efficientnet-b7


100%|██████████| 157/157 [1:54:31<00:00, 34.96s/it]


In [11]:
a = submission.copy()

In [12]:
a

,index,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
0,50000,5.333109,0.000786,5.154987,0.799258,5.333321e+00,5.333334,0.004966,5.333316,5.332681,...,0.015342,0.117381,0.686799,5.168938,5.333252,2.353002,2.114237,5.276643,0.000249,1.461977
1,50001,0.079933,5.333024,4.308621,0.321108,5.333333e+00,0.055538,3.873186,0.004278,5.332883,...,0.010941,2.659327,0.433471,0.001455,5.331760,5.271769,0.001169,0.000038,0.020789,0.000369
2,50002,0.002084,0.047081,0.534344,4.854992,5.332068e+00,0.082263,5.333325,0.513914,5.333334,...,1.293147,5.332219,5.333324,0.002918,5.333326,0.028079,5.333323,0.000747,0.000775,5.333291
3,50003,5.332211,5.333328,0.116671,3.375474,5.674926e-01,5.281528,2.926519,0.077680,5.048617,...,0.365491,1.812927,2.095199,5.333324,0.000584,0.707203,0.010548,5.333333,0.385435,5.333322
4,50004,0.369131,0.000648,5.333243,0.850174,5.333329e+00,5.333333,0.003444,0.045627,0.000375,...,5.333310,0.015040,5.333319,5.333333,5.327534,0.095970,5.333309,0.001864,2.158033,0.000316
5,50005,0.034048,4.685974,5.288535,0.000027,6.023451e-03,3.205131,0.043479,0.001268,0.552581,...,5.331413,0.003334,0.002417,0.035788,4.553675,0.770944,5.332807,0.173408,5.333319,0.700680
6,50006,0.003265,2.995220,5.333170,1.596542,5.332359e+00,0.775923,1.075713,0.008143,0.243012,...,5.266951,0.121249,4.962503,5.333332,5.333319,0.047059,5.222156,5.142009,0.028600,1.336614
7,50007,2.780594,0.000045,5.333291,0.008446,5.333334e+00,0.000287,0.000002,5.333017,0.000077,...,1.692154,0.020030,0.000012,0.126785,0.001313,5.329641,5.333330,5.289057,0.000891,0.000010
8,50008,5.308982,0.000927,4.973888,0.002305,5.333326e+00,3.328396,4.820608,0.042110,5.333292,...,1.678420,5.333326,1.749784,0.186473,2.397929,5.333211,4.867041,0.000043,0.114272,5.333334
9,50009,5.333314,0.000410,5.202284,5.333311,9.687143e-05,1.660781,3.505067,5.333232,2.976702,...,5.333308,5.333244,0.004187,4.750575,4.593034,0.000966,1.132119,0.026243,0.171000,5.330060


In [ ]:
submission = pd.read_csv('Output/sample_submission.csv')

In [20]:
submission.iloc[:,1:] = np.where(a.values[:,1:] / 5>=0.7, 1,0)

In [19]:
submission

,index,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
0,50000,1,0,1,0,1,1,0,1,1,...,0,0,0,1,1,0,0,1,0,0
1,50001,0,1,1,0,1,0,1,0,1,...,0,0,0,0,1,1,0,0,0,0
2,50002,0,0,0,1,1,0,1,0,1,...,0,1,1,0,1,0,1,0,0,1
3,50003,1,1,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,1,0,1
4,50004,0,0,1,0,1,1,0,0,0,...,1,0,1,1,1,0,1,0,0,0
5,50005,0,1,1,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0
6,50006,0,0,1,0,1,0,0,0,0,...,1,0,1,1,1,0,1,1,0,0
7,50007,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,1,1,1,0,0
8,50008,1,0,1,0,1,0,1,0,1,...,0,1,0,0,0,1,1,0,0,1
9,50009,1,0,1,1,0,0,1,1,0,...,1,1,0,1,1,0,0,0,0,1


In [17]:
submission.to_csv('./Output/EfficientNetB7-fold5Ensemble_TTA_07.csv', index=False)

In [148]:
model_output

tensor([[8.8889e-01, 8.3333e-02, 0.0000e+00, 7.7778e-01, 8.3333e-02, 4.1667e-01,
         4.7319e-01, 6.9444e-01, 1.0000e+00, 0.0000e+00, 9.7222e-01, 1.6667e-01,
         1.0000e+00, 6.9444e-01, 8.3333e-01, 9.7222e-01, 9.4444e-01, 4.7222e-01,
         2.5000e-01, 9.4444e-01, 2.7778e-01, 8.8520e-01, 6.3889e-01, 9.1667e-01,
         6.1111e-01, 5.5556e-02],
        [8.6111e-01, 2.5000e-01, 1.1111e-01, 7.5000e-01, 5.5556e-02, 6.1111e-01,
         6.6739e-01, 8.3334e-01, 1.0000e+00, 2.7778e-02, 1.0000e+00, 1.3889e-01,
         9.1667e-01, 6.9444e-01, 9.4444e-01, 1.0000e+00, 8.3333e-01, 6.6654e-01,
         1.6667e-01, 1.0000e+00, 8.3333e-02, 1.0000e+00, 6.1111e-01, 1.0000e+00,
         8.6111e-01, 2.7778e-02],
        [9.1667e-01, 3.8897e-01, 8.3333e-02, 8.3333e-01, 5.5556e-02, 4.7222e-01,
         4.7218e-01, 7.7778e-01, 1.0000e+00, 8.3333e-02, 9.7222e-01, 1.6667e-01,
         9.4444e-01, 7.5000e-01, 9.1667e-01, 9.7222e-01, 8.0556e-01, 4.4444e-01,
         1.6667e-01, 1.0000e+00, 1.6666e-